# Initialize qcodes

In [1]:
import qcodes as qc
from qcodes.dataset.sqlite.database import initialise_or_create_database_at
import IPython.lib.backgroundjobs as bg
from plottr.apps import inspectr
from qcodes.instrument.parameter import Parameter
from qc_measure_20240710 import *
print_versions()

qcodes: 0.46.0
qcodes_contrib_drivers: 0.22.0
plottr: 0.13.0
python: 3.12.3 | packaged by conda-forge | (main, Apr 15 2024, 18:20:11) [MSC v.1938 64 bit (AMD64)]
matplotlib: 3.8.4
numpy: 1.26.4
scipy: 1.14.0


Update db_path here:

In [2]:
# Create a new db file name for each cooldown.
db_path = 'test_20240707_XL105_InAs2DEGJJ15-20221222-cooldown4_username.db'

qc.logger.start_all_logging()
initialise_or_create_database_at(db_path)

station = qc.Station()

# Open Plottr
jobs = bg.BackgroundJobManager()
jobs.new(inspectr.main, db_path)

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\Measurement\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\Measurement\.qcodes\logs\240710-12248-qcodes.log


<BackgroundJob #0: <function main at 0x000002425DC218A0>>

# Load instruments

In [3]:
from qcodes_contrib_drivers.drivers.QDevil import QDAC2
# qdac.close()
qdac = QDAC2.QDac2('QDAC',  address= 'COM4')
qdac.timeout(1)

from qcodes.instrument_drivers.Keysight.Keysight_34461A_submodules import Keysight_34461A
# dmm.close()
dmm =  Keysight_34461A('Keysight_34461A', 'USB0::0x2A8D::0x1301::MY60060848::INSTR')
dmm.timeout(1)

Connected to: QDevil QDAC-II (serial:78, firmware:7-0.17.5) in 0.15s
Connected to: Keysight Technologies 34461A (serial:MY60060848, firmware:A.03.03-03.15-03.03-00.52-04-03) in 0.09s


# Configure Parameters

In [4]:
def get_V_measure():
    return dmm.volt()

V_measure = Parameter('V_measure', unit='V', label='Vmeas', 
                                 get_cmd=get_V_measure, set_cmd=None)


Dummy_x = DummyParameter(name='dummy_x',unit='')

Dummy_y = DummyParameter(name='dummy_y',unit='')

Vg = qdac.ch01.dc_constant_V

# Configure Station

Run the cell below to check instruments and parameters in the station

In [5]:
station.components.keys()

dict_keys([])

In [6]:
st_components = list(station.components.keys())
for i in st_components:
    station.remove_component(i)

for i in [dmm, qdac, V_measure, Vg]:
    print(i)
    station.add_component(i)

<Keysight_34461A: Keysight_34461A>
<QDac2: QDAC>
V_measure
QDAC_ch01_dc_constant_V


# Begin scans

## 1d

In [7]:
qm = QMeasure(exp_name='20240707-Al-InAs-wire-15',sample_name='left-12-13-a')
qm.set_parameters_to_aquire(para_list=[V_measure], parallel=True)
qm.set_parameters_in_measurement_name(para_list=[V_measure, dmm.volt])

qm.set_scan_parameter('x', para=Dummy_x, start=0, stop=1, points=101, delay=0.01)
# qm.set_scan_parameter('y', para=None, start=None, stop=None, points=None, delay=0)
qm.scan2d(bwd=True)

Starting experimental run with id: 39. 
Starting experimental run with id: 40. 


Total: 0 min 10 s. Each point: 0.045 s.
DAT and SET file saved: DAT\20240707-Al-InAs-wire-15.left-12-13-a.39
DAT and SET file saved: DAT\20240707-Al-InAs-wire-15.left-12-13-a.40


In [8]:
qm = QMeasure(exp_name='20240707-Al-InAs-wire-15',sample_name='left-12-13-a')
qm.set_parameters_to_aquire(para_list=[V_measure], parallel=True)
qm.set_parameters_in_measurement_name(para_list=[V_measure, dmm.volt])

qm.set_scan_parameter('x', para=Vg, start=0, stop=0.1, points=101, delay=0.01)
# qm.set_scan_parameter('y', para=None, start=None, stop=None, points=None, delay=0)
qm.scan2d(bwd=True)

Starting experimental run with id: 41. 
Starting experimental run with id: 42. 


Total: 0 min 9 s. Each point: 0.041 s.
DAT and SET file saved: DAT\20240707-Al-InAs-wire-15.left-12-13-a.41
DAT and SET file saved: DAT\20240707-Al-InAs-wire-15.left-12-13-a.42


## 2d

In [9]:
qm = QMeasure(exp_name='20240707-Al-InAs-wire-15',sample_name='left-12-13-a')
qm.set_parameters_to_aquire(para_list=[V_measure], parallel=True)
qm.set_parameters_in_measurement_name(para_list=[V_measure, dmm.volt])

qm.set_scan_parameter('x', para=Vg, start=0, stop=0.1, points=101, delay=0.01)
qm.set_scan_parameter('y', para=Dummy_y, start=0, stop=1, points=10, delay=0.1)
qm.scan2d(bwd=True)

Starting experimental run with id: 43. 
Starting experimental run with id: 44. 


Total: 1 min 27 s. Each point: 0.043 s.
DAT and SET file saved: DAT\20240707-Al-InAs-wire-15.left-12-13-a.43
DAT and SET file saved: DAT\20240707-Al-InAs-wire-15.left-12-13-a.44


## Export data manually

In [10]:
d2d = Db2Dat()
exp_id = 44
dat_folder = r'.\DAT'
dat_path = d2d.to_dat(db_path, exp_id, dat_folder,overwrite=False)

File ".\DAT\20240707-Al-InAs-wire-15.left-12-13-a.44.dat" already exists and has not been overwritten.
File ".\DAT\20240707-Al-InAs-wire-15.left-12-13-a.44.set" already exists and has not been overwritten.
DAT and SET file saved: .\DAT\20240707-Al-InAs-wire-15.left-12-13-a.44


# Special scans: fast1d

In [11]:
from qc_fast1d_20240710 import *

In [12]:
def Ibias_to_Vdac(x):
    y =  x/1.
    return y

# You may want to modify the output_channel and nplc
TRG_DAC = TrigQDAC(qdac,ch=1,trg_out=1,trg_in=1,trg_internal=1,)
TRG_DMM = TrigDMM(dmm,nplc=0.2) 

def reset_trig_modes():
    '''
    Reset trig modes of dac and dmm
    '''
    TRG_DAC.back_to_no_trig()
    TRG_DMM.back_to_auto_trig()
    
class QMeasureFast(QMeasure):
    '''
    A new classs replacing _scan1d in QMeasure, for fast scan with triggers
    '''  
    def _scan1d(self,datasaver, list_x, y):
        para_x = self.scan_para['x']['parameter']
        delay_x = self.scan_para['x']['delay']
        parameters_to_fetch = self.monitor_para['list']
        para_y = self.scan_para['y']['parameter']

        list_v = Ibias_to_Vdac(list_x)
        # make sure that parameters_to_fetch == [Vmeasure]
        p_fetch = parameters_to_fetch[0]

        if self.counter==0:
            # Configure the DMM buffer and trigger
            TRG_DMM.initialize_meas(list_v,trig_delay=delay_x)
            # This will upload the sweeping list and configure triggers
            TRG_DAC.initialize_sweep(list_v)
            self.counter += 1

        # Send the initial trigger
        TRG_DMM.enter_meas()
        TRG_DAC.fire_sweep() 

        try:
            d = TRG_DMM.fetch()
        except KeyboardInterrupt:
            raise(KeyboardInterrupt)

        d = d[1:]
        list_val = [[(p_fetch, i)] for i in d]

        for x, val in zip(list_x, list_val):
            rslt = [(para_y, y), (para_x, x)] + val
            datasaver.add_result(*rslt)

In [13]:
qm_fast = QMeasureFast(exp_name='20240707-Al-InAs-wire-15',sample_name='left-12-13-a')
# Ibias_fast must be a dummy parameter, not the real Ibias parameter, only the parameter name will be used
Ibias_fast = DummyParameter(name='Ibias_fast', unit='uA')

qm_fast.set_parameters_to_aquire(para_list=[V_measure], parallel=True)
qm_fast.set_parameters_in_measurement_name(para_list=[V_measure, dmm.volt])

qm_fast.set_scan_parameter('x', para=Ibias_fast, start=0, stop=1, points=101, delay=0.01)
# qm_fast.set_scan_parameter('y', para=Dummy_y, start=0, stop=1, points=2, delay=0)
qm_fast.scan2d(bwd=True)

reset_trig_modes()

Starting experimental run with id: 45. 
Starting experimental run with id: 46. 
Total: 0 min 4 s. Each point: 0.015 s.
DAT and SET file saved: DAT\20240707-Al-InAs-wire-15.left-12-13-a.45
DAT and SET file saved: DAT\20240707-Al-InAs-wire-15.left-12-13-a.46
Setting qdac back to no trigger and fixed voltage...
Setting dmm back to auto trig mode (if failed, run it once more)...


In [18]:
qm_fast = QMeasureFast(exp_name='20240707-Al-InAs-wire-15',sample_name='left-12-13-a')
# Ibias_fast must be a dummy parameter, not the real Ibias parameter, only the parameter name will be used
Ibias_fast = DummyParameter('Ibias_fast',unit='uA')

qm_fast.set_parameters_to_aquire(para_list=[V_measure], parallel=True)
qm_fast.set_parameters_in_measurement_name(para_list=[V_measure, dmm.volt])

qm_fast.set_scan_parameter('x', para=Ibias_fast, start=0, stop=1, points=101, delay=0)
qm_fast.set_scan_parameter('y', para=Dummy_y, start=0, stop=1, points=10, delay=0)
qm_fast.scan2d(bwd=True)

reset_trig_modes()

Starting experimental run with id: 49. 
Starting experimental run with id: 50. 


Total: 0 min 10 s. Each point: 0.005 s.
DAT and SET file saved: DAT\20240707-Al-InAs-wire-15.left-12-13-a.49
DAT and SET file saved: DAT\20240707-Al-InAs-wire-15.left-12-13-a.50
Setting qdac back to no trigger and fixed voltage...
Setting dmm back to auto trig mode (if failed, run it once more)...


## Reset trigger mode

In [19]:
reset_trig_modes()

Setting qdac back to no trigger and fixed voltage...
Setting dmm back to auto trig mode (if failed, run it once more)...
